In [1]:
import numpy as np
import pylab as pl
import arrayfire as af
import h5py
af.set_backend("cpu")
%matplotlib inline

In [2]:
# Setting velocity and spatial grid points
N_positions = 1048
ghost_zones = 3
N_velocity  = N_positions

In [3]:
# Boundaries of domain
left_boundary  = 0
right_boundary = 1.0
length         = right_boundary - left_boundary

In [4]:
# Setting mass of the particle, boltzmann-constant
mass_particle      = 1.0
boltzmann_constant = 1.0
charge_particle    = -11.0

In [5]:
# Scattering time scale
tau   = np.inf #0.01
# Magnitude of maximum velocity
v_max = 5

In [6]:
# Setting the parameters for time here
N_t        = 3000
final_time = 3
dt         = final_time/N_t
time_array = np.linspace(dt, final_time, N_t)

In [7]:
# Setting up of spatial and velocity grids:
x  = np.linspace(left_boundary, right_boundary, N_positions)
dx = x[1] - x[0]

In [8]:
# Obtaining the coordinates for the ghost-zones:
x_ghost_left  = np.linspace(-(ghost_zones)*dx + left_boundary, left_boundary - dx, ghost_zones)
x_ghost_right = np.linspace(right_boundary + dx, right_boundary + ghost_zones*dx , ghost_zones)

In [9]:
# Combining them to obtain the entire spatial grid
x  = np.concatenate([x_ghost_left, x, x_ghost_right])

In [10]:
# Obtaining the velocity grid
v  = np.linspace(-v_max, v_max, N_velocity)
x  = af.Array.as_type(af.to_array(x), af.Dtype.f64)
v  = af.Array.as_type(af.to_array(v), af.Dtype.f64)

In [11]:
# Conversion to allow for easy vectorization
x = af.tile(x, 1, N_velocity)
v = af.tile(af.reorder(v), N_positions + 2*ghost_zones, 1)

In [12]:
def calculate_density(f, v):
    deltav           = af.sum(v[0, 1]-v[0, 0])
    value_of_density = af.sum(f, 1)*deltav
    
    af.eval(value_of_density)
    
    return(value_of_density)

In [13]:
def f_interp(dt, x, v, f):

    x_new     = x - 0.5*v*dt
    step_size = af.sum(x[1,0] - x[0,0])
    
    f_interp  = af.constant(0, N_positions + 2*ghost_zones, N_velocity, dtype=af.Dtype.f64)
    
    # Interpolating:
    
    x_temp = x_new[ghost_zones:-ghost_zones, :]
    
    while(af.sum(x_temp<left_boundary)!=0):
        x_temp = af.select(x_temp<left_boundary,
                           x_temp + length,
                           x_temp
                          )
        
    while(af.sum(x_temp>right_boundary)!=0):
        x_temp = af.select(x_temp>right_boundary,
                           x_temp - length,
                           x_temp
                          )

    x_interpolant = x_temp/step_size + ghost_zones
    
    f_interp[ghost_zones:-ghost_zones, :] = af.approx1(f, x_interpolant,\
                                                       af.INTERP.CUBIC_SPLINE
                                                      )
#     pl.contourf(np.array(x[3:-3]), np.array(v[3:-3]), np.array(f_current[3:-3] - f_background[3:-3]), 100)
#     pl.colorbar()
#     pl.xlabel('$x$')
#     pl.ylabel('$v$')
#     pl.title('Time = ' + str(t0))
#     pl.savefig('images/' + "%04d"%time_index + '.png', dpi = 300)
#     pl.clf()
    
    af.eval(f_interp)
    return f_interp

In [14]:
def f_interp_v(dt, x, v, f, E):
    v_new     = v - charge_particle * dt * af.tile(E, 1, N_velocity)
    step_size = af.sum(v[0,1] - v[0,0])
    
    # Interpolating:
       
    v_interpolant = (v_new + v_max)/step_size

    f_interp      = af.approx1(af.reorder(f),\
                               af.reorder(v_interpolant),\
                               af.INTERP.CUBIC_SPLINE
                              )
    
    f_interp          = af.reorder(f_interp)
    
    af.eval(f_interp)

    return f_interp

In [15]:
def fft_poisson(rho):
    
    if((af.Array.elements(rho)%2)==0):
        kspace = af.to_array(np.append(np.arange(0, (N_positions - 1)/2 - 1),\
                                       np.arange(-((N_positions - 1)/2) - 1, 0)
                                      )
                            )
    else:
        kspace = af.to_array(np.append(np.arange(0, N_positions/2 - 1),\
                                       np.arange(-N_positions/2, 0)
                                      )
                            )
        
        
    rho_kspace = af.fft(rho)
    V_kspace   = af.constant(0, af.Array.elements(rho), dtype=af.Dtype.c64)
    kspace     = af.Array.as_type(kspace, af.Dtype.c64)
    
    V_kspace[1:] =  (1/(4 * np.pi**2 * kspace[1:]**2)) * rho_kspace[1:]
    V_kspace[0]  =  0

    E_kspace     =  -1j * 2 * np.pi * kspace * V_kspace
    
    E = af.ifft(E_kspace)
    
    return E

In [16]:
# Intializing the values for f:
rho0           = 1.0
delta_rho_real = 5e-3
delta_rho_imag = 0

rho       = rho0 + (delta_rho_real * af.cos(2*np.pi*x) - delta_rho_imag * af.sin(2*np.pi*x))

T0      = 1.0
delta_T = 0 

T         = T0 - (delta_T) * (af.cos(2*np.pi*x) + af.sin(2*np.pi*x))

f_initial = rho * af.sqrt(mass_particle/(2*np.pi*boltzmann_constant*T)) * \
            af.exp(-mass_particle*v**2/(2*boltzmann_constant*T))

In [17]:
f_initial[:ghost_zones,:]   = f_initial[-(2*ghost_zones + 1):-(ghost_zones + 1)]
f_initial[-ghost_zones:, :] = f_initial[(ghost_zones + 1):(2*ghost_zones + 1)]

In [ ]:
f_current = f_initial
data = np.zeros(time_array.size)
f_background = rho0 * af.sqrt(mass_particle/(2*np.pi*boltzmann_constant*T)) * \
               af.exp(-mass_particle*v**2/(2*boltzmann_constant*T))

In [ ]:
for time_index, t0 in enumerate(time_array):
    if(time_index%100==0):
        print("Physical Time            = ", t0)
    # We shall split the Boltzmann-Equation and solve it:
    # In this step we are solving the collisionless equation
    fstar = f_interp(dt, x, v, f_current)

    fstar[:ghost_zones,:]                = fstar[-(2*ghost_zones + 1):-(ghost_zones + 1)]
    fstar[N_positions + ghost_zones:, :] = fstar[(ghost_zones + 1):(2*ghost_zones + 1)]
    
    E       = af.constant(0, N_positions + 2*ghost_zones, dtype=af.Dtype.c64)
    E_local = fft_poisson(charge_particle*(calculate_density(fstar, v)[3:-4] - 1))
    E_local = af.join(0, E_local, E_local[0])
    
    E[ghost_zones:-ghost_zones]   = E_local
    E[:ghost_zones]               = E[-(2*ghost_zones + 1):-(ghost_zones + 1)]
    E[N_positions + ghost_zones:] = E[(ghost_zones + 1):(2*ghost_zones + 1)]
    
    fdoublestar = f_interp_v(dt, x, v, fstar, af.real(E))
    
    fdoublestar[:ghost_zones,:]                = fdoublestar[-(2*ghost_zones + 1):-(ghost_zones + 1)]
    fdoublestar[N_positions + ghost_zones:, :] = fdoublestar[(ghost_zones + 1):(2*ghost_zones + 1)]
    
    f_final = f_interp(dt, x, v, fdoublestar)
    
    f_final[:ghost_zones,:]                = f_final[-(2*ghost_zones + 1):-(ghost_zones + 1)]
    f_final[N_positions + ghost_zones:, :] = f_final[(ghost_zones + 1):(2*ghost_zones + 1)]
    
    f_current = f_final
    
    data[time_index] = af.max(calculate_density(f_current, v))

Physical Time            =  0.001


In [ ]:
# f_diff = np.array(f_current[3:-3] - f_background[3:-3])
# v_pl = np.array(af.reorder(v[0]))
# h5f    = h5py.File("LT_2048.h5", 'r')
# data_2 = h5f['density'][:]
# time_2 = h5f['time'][:]
# h5f.close()
# pl.semilogy(time_array, data - 1)
# pl.semilogy(time_2, data_2)
# pl.ylabel(r'$f_{\mathrm{diff}}$')
# pl.xlabel('$v$')
# pl.title(r'At $x=0$')

In [ ]:
h5f = h5py.File('CK_' + str(N_positions) + '.h5', 'w')
h5f.create_dataset('f_dist', data = f_current)
h5f.close()

In [ ]:
N_positions

In [ ]:
pl.semilogy(data-1)